# Word embedding and PCA

In [ ]:
# import packages
import pandas as pd
import os
import math
import numpy as np
from nltk import ngrams
from matplotlib import pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [ ]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import parameters as params
import dataframe_columns as df_col

## Predefined functions:

## Main program:

In [ ]:
# Check if my words are within the BERT vocabulary
from transformers import BertTokenizer

# Load the BERT tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# Your list of words to check
words_to_check = tokenizer(text)

# Check if each word is in the BERT vocabulary
for word in words_to_check:
    if word in tokenizer.get_vocab():
        # print(f"'{word}' is in the BERT vocabulary.")
        pass
    else:
        print(word)

In [ ]:
# Word embedding
from transformers import BertTokenizer, BertModel
import torch

# Load the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

def text_embedding(text_500):
    # Split the document into sentences
    sentences = sent_tokenize(text_500)  # You may need to use a more robust sentence tokenizer

    # Initialize a list to store sentence embeddings
    sentence_embeddings = []

    # Tokenize and embed each sentence
    for sentence in sentences:
        tokens = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**tokens)
            sentence_embedding = torch.mean(outputs.last_hidden_state, dim=1)
        sentence_embeddings.append(sentence_embedding)

    # Average pooling to obtain a single vector for the entire document
    text_embedding = torch.mean(torch.stack(sentence_embeddings), dim=0)
    
    return text_embedding

In [ ]:
# Iterate through the rows of the dataframe and embed the text
input_path = fpath.poten_litera_db_kw_count
output_path = fpath.poten_litera_db_kw_count_with_embedding

df = pd.read_csv(input_path, header=0, sep=',')
df.columns = df_col.db_ranked_columns

# clear file
plib.clear_file(output_path)

for ind in df.index:
    index = int(df.at[ind, "INDEX"])
    
    
    text_500 = df.at[ind, "TEXT_500"]
    text_500 = plib.process_text(text_500, lower=True)
    text_embedding = text_embedding(text_500)
    df.at[ind, "TEXT_EMBEDDING"] = text_embedding
    
    print("ind:", ind, "index:", df.at[ind, "INDEX"])
    print("\n")
    
# Save the dataframe
df.to_csv(fpath.output_path, index=False)

# Read the dataframe